# Final Project: Interactive Connect 4
*AI move determination using Minimax algorithm with Alpha-beta pruning*

Python Pygame Implementation of the Connect 4 game depicted below. This has been generalized to any $n \times m$ board

<img src="https://cdn.dribbble.com/users/325367/screenshots/1145503/final_out.jpg" width="400" height="400">

---

## 1. Package Importation

In [1]:
#Import packages needed to build connect 4 pygame
import pygame
import numpy as np
import random
import sys
import math
from copy import deepcopy

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## 2. Variable Definitions

In [2]:
#Define python/pygame colours to build connect four board
RED    = (255,0,0)
YELLOW = (255,255,0)
BLUE   = (0,42,243)
BLACK  = (0,0,0)

In [3]:
#Define disc states to differentiate between player discs, AI discs and empty positions
EMPTY_SLOT  = 0
PLAYER_DISC = 1
AI_DISC     = 2

#Define player and AI turns
PLAYER = 0
AI     = 1

## 3. Building the Connect 4 Class

In [4]:
class ConnectFour:
    
    """
    Class defining connect four board and methods needed to build an adversarial game against an AI using 
    minimax with alpha-beta pruning
    
    Adapted from Galli, K. (2019) KeithGalli/Connect4-Python [Python]. Retrieved from 
    https://github.com/KeithGalli/Connect4-Python. Improvements to the code include an object-oriented implementation
    of the ConnectFour board and its properties/attributes resulting in a neater and organized code. Improved
    flexibility and generalizability of code, allowing the game to be expanded to larger board sizes.
    
    Alpha-Beta Pruning with Minimax adapted from Wikipedia (2002). Alpha–beta pruning Pseudocode. 
    Retrieved from https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning#Pseudocode
    
    """
    
    def __init__(self,rows=6,columns=7):
        
        """
        Initialize board attributes
        """
        
        self.rows= rows
        self.columns= columns
        self.board= np.zeros((rows,columns))
    
    def drop_disc(self, row, col, disc):
        
        #Drop disc in specified location
        
        self.board[row][col] = disc
        
    def is_valid_location(self, col):
        
        #Check to see if column is not entirely filled (i.e. if there are empty slots)
        return self.board[self.rows-1][col] == 0
    
    def get_vacant_row(self,col):
        
        #Return vacant row
        #Note that the topmost vacant row will be returned
        #This will be rectified due to the way the board is printed in function below
        
        for row in range(self.rows):
            if self.board[row][col] == 0:
                return row
        
    def print_board(self):
        
        #The board is printed in reverse order (from bottom row to top)
        #due to the nature of the indexing utilized
        print(np.flip(self.board, 0))
    
    def get_valid_positions(self):
        
        """
        Determine which columns a disc can be dropped into by players
        """
        
        #Initialize empty list to store valid positions
        valid_positions= []
        
        for col in range(self.columns):
            if self.is_valid_location(col):
                valid_positions.append(col)
            
        return valid_positions
    
    
    
    def winning_move(self, disc):
        
        # Check horizontal locations for win
        #All columns except the last three are checked because these are the only possible starting
        #positions for a winning move that lead to four horizontal disc
        for col in range(self.columns-3):
            
            #Iterating over all the rows
            for row in range(self.rows):
                if self.board[row][
                    col] == disc and self.board[row][col+1] == disc and self.board[
                    row][col+2] == disc and self.board[row][col+3] == disc:
                    return True

        #Check vertical locations for win
        for col in range(self.columns):
            
            #Only the first three rows are checked since the starting position for a winning move 
            #can only be from those rows to have a line of four
            for row in range(self.rows-3):
                if self.board[row][
                    col] == disc and self.board[row+1][
                    col] == disc and self.board[row+2][col] == disc and self.board[row+3][col] == disc:
                    return True

        # Check positive diagonal locations for a winning configuration
        
        #Starting position for a winning move are all rows and columns except the last three of each
        for col in range(self.columns-3):
            for row in range(self.rows-3):
                if self.board[row
                             ][col] == disc and self.board[row+1
                                                          ][col+1] == disc and self.board[row+2][col+2] == disc and self.board[row+3][col+3] == disc:
                    return True

                
        # Check negative diagonal locations for a winning configuration
        
        #Starting position for a winning move are all rows and columns except the last three columns
        #and first three rows
        for col in range(self.columns-3):
            for row in range(3, self.rows):
                if self.board[row][col] == disc and self.board[row-1][col+1] == disc and self.board[row-2][col+2] == disc and self.board[row-3][col+3] == disc:
                    return True

                
    def evaluate_window(self,window, disc):
        
        """
        
        Assigning score to players depending on the state of the inputted window
        """
        
        #Initial score to 0
        score = 0
        
        #If the current disc is the player's, set opposing disc to AI disc
        if disc == PLAYER_DISC:
            opp_disc = AI_DISC
        else:
            opp_disc = PLAYER_DISC
        
        #If a player has successfully connected four discs, add 100 to score
        if window.count(disc) == 4:
            score += 100
            
        #If a player has successfully connected three discs, add 100 to score
        elif window.count(disc) == 3 and window.count(EMPTY_SLOT) == 1:
            score += 5
            
        #If a player has successfully connected two discs, add 100 to score
        elif window.count(disc) == 2 and window.count(EMPTY_SLOT) == 2:
            score += 2
        
        #If the opposing player has successfully connected three discs, subtract 4 from score
        if window.count(opp_disc) == 3 and window.count(EMPTY_SLOT) == 1:
            score -= 4
        
        #Return the score given the input window
        return score
    
    def score_position(self, disc, window_length=4):
        
        """
        Score player based on board configuration using the scoring system established 
        in the evaluate_window function
        """
        
        #Initialize score to 0
        score = 0

        ## Score center column
        center_window = [int(i) for i in list(self.board[:, self.columns//2])]
        score += center_window.count(disc) * 3

        ## Score rows
        for row in range(self.rows):
            
            #Get all the columns of the particular row
            row_window = [int(i) for i in list(self.board[row,:])]
            
            #For all the columns except the last three in the given row
            for col in range(self.columns-3):
                
                #Calculate the score using the scoring system
                window = row_window[col:col+window_length]
                score += self.evaluate_window(window, disc)

        ## Score verticals (column discs configurations)
        for col in range(self.columns):
            col_window = [int(i) for i in list(self.board[:,col])]
            for row in range(self.rows-3):
                window = col_window[row:row+window_length]
                score += self.evaluate_window(window, disc)

        ## Score positive diagonals
        for row in range(self.rows-3):
            for col in range(self.columns-3):
                window = [self.board[row+i][col+i] for i in range(window_length)]
                score += self.evaluate_window(window, disc)
        
        #Score negative diagonals
        for row in range(self.rows-3):
            for col in range(self.columns-3):
                window = [self.board[row+3-i][col+i] for i in range(window_length)]
                score += self.evaluate_window(window, disc)

        return score
    
    
    def terminal_node(self):
        
        """
        Check if a terminal state has been reached
        """
        
        #Check if a player has successfully connected four discs or 
        #if there are no other empty slots on the board
        return self.winning_move(PLAYER_DISC) or self.winning_move(AI_DISC) or len(self.get_valid_positions()) == 0
    
    def alpha_beta_minimax_search(self, depth, alpha, beta, maximizingPlayer):
        
        """
        Evaluate the best branches possible going as deep down the tree as specified by depth
        """
        
        #Determine possible columns that can played into given current board state
        possible_moves= self.get_valid_positions()
        
        if depth == 0 or self.terminal_node():
            if self.terminal_node():
                if self.winning_move(AI_DISC):
                    return (None, 50000)
                elif self.winning_move(PLAYER_DISC):
                    return (None, -50000)
                else: # Game is over, no more valid moves
                    return (None, 0)
            else: # Depth is zero
                #Return board heurisitic value
                return (None, self.score_position(AI_DISC))
            
        if maximizingPlayer:
            value = -float("inf")
            move= np.random.choice(possible_moves)
            
            for col in possible_moves:
                row= self.get_vacant_row(col)
                next_state= deepcopy(self)
                next_state.drop_disc(row,col,AI_DISC)
                new_score = next_state.alpha_beta_minimax_search(depth-1, alpha, beta, False)[1]
                
                #Update value to highest score encountered based on all explored moves
                if new_score > value:
                    value= new_score
                    move= col
                
                alpha= max(value,alpha)
                
                if alpha >= beta:
                    break
            
            return move,value
        
        #Minimizing player trying to obtain the lowest score
        else:
    
            value = float("inf")
            move= np.random.choice(possible_moves)
            
            for col in possible_moves:
                row= self.get_vacant_row(col)
                next_state= deepcopy(self)
                next_state.drop_disc(row,col,PLAYER_DISC)
                new_score = next_state.alpha_beta_minimax_search(depth-1, alpha, beta, True)[1]
                
                #Update value to highest score encountered based on all explored moves
                if new_score < value:
                    value= new_score
                    move= col
                
                beta= min(value,beta)
                
                if alpha >= beta:
                    break
            
            return move,value
        

## 4. Pygame Interface

In [5]:
def draw_pygame_board(board):
        
        """
        Draw the current ConnectFour board in pygame 
        
        Input:
        - board: A ConnectFour class which contains an attribute for the board and possible actions
        
        """    
        
        #Iterating over all the columns of the inputted board
        for col in range(board.columns):
            
            #Iterating over all the rows of the inputted board
            for row in range(board.rows):
                
                #Draw a rectangle and a circle inside it to build up a visualization of an empty ConnectFour board
                pygame.draw.rect(screen, BLUE, (col*TILESIZE, row*TILESIZE+TILESIZE, TILESIZE, TILESIZE))
                pygame.draw.circle(screen, BLACK, (int(col*TILESIZE+TILESIZE/2), int(row*TILESIZE+TILESIZE+TILESIZE/2)), RADIUS)
        
        #Iterate through every row and column
        for col in range(board.columns):
            for row in range(board.rows):
                
                #If the disc in the current board position is the player's, replace with a red disc
                if board.board[row][col] == PLAYER_DISC:
                    pygame.draw.circle(screen, RED, (int(col*TILESIZE+TILESIZE/2), height-int(row*TILESIZE+TILESIZE/2)), RADIUS)
                
                #If the disc in the current board position is the AI's, replace with a yellow disc
                elif board.board[row][col] == AI_DISC: 
                    pygame.draw.circle(screen, YELLOW, (int(col*TILESIZE+TILESIZE/2), height-int(row*TILESIZE+TILESIZE/2)), RADIUS)
        pygame.display.update()
        

In [6]:
#Initialize Connect Four board and print the empty board
#Note parameter (row,column) can be changed to create boards larger than the default 6 by 7
board = ConnectFour(6,7)

#Initialize game_over to False to signal that the game has begun and is still in progress
game_over = False

pygame.init()

#Initialize attributes for the pygame board
TILESIZE = 100
width = board.columns * TILESIZE
height = (board.rows+1) * TILESIZE

#Specify size of pygame window
size = (width, height)

#Specify radius size for player discs given size of squares
RADIUS = int(TILESIZE/2 - 5)

screen = pygame.display.set_mode(size)

draw_pygame_board(board)
pygame.display.update()

pygame.font.init()
game_font = pygame.font.SysFont('helvetica', 75)

#Randomly choose who starts first
turn = random.randint(PLAYER, AI)

while not game_over:

        #If the game is quit, exit
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()


        #If the mouse is moved, move the disc along with it
        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, BLACK, (0,0, width, TILESIZE))
            xpos = event.pos[0]
            if turn == PLAYER:
                pygame.draw.circle(screen, RED, (xpos, int(TILESIZE/2)), RADIUS)

        #Update the display to see changes
        pygame.display.update()

        #If the mouse is clicked (to drop)
        if event.type == pygame.MOUSEBUTTONDOWN:
            pygame.draw.rect(screen, BLACK, (0,0, width, TILESIZE))

            # If it is the player's turn
            if turn == PLAYER:

                #Get the drop position and use it to estimate the intended board column
                xpos = event.pos[0]
                col = int(math.floor(xpos/TILESIZE))

                #If the selected column is a valid column
                if board.is_valid_location(col):

                    #Drop the disc down into the next available row
                    row = board.get_vacant_row(col)
                    board.drop_disc(row, col, PLAYER_DISC)

                    #If the drop results in successfully connecting four discs
                    if board.winning_move(PLAYER_DISC):
                        #Player 1 Wins
                        label = game_font.render("Player 1 wins!!", 1, RED)
                        screen.blit(label, (40,10))
                        game_over = True

                    turn += 1
                    turn = turn % 2

                    #board.print_board()
                    draw_pygame_board(board)

            #Ask for Player 2 Input
            if turn == AI and not game_over:

                col, minimax_score = board.alpha_beta_minimax_search(4, -float('inf'), float('inf'), True)

                if board.is_valid_location(col):
                    row = board.get_vacant_row(col)
                    board.drop_disc(row, col, AI_DISC)

                    if board.winning_move(AI_DISC):
                        label = game_font.render("AI wins!!", 1, YELLOW)
                        screen.blit(label, (40,10))
                        game_over = True

                    draw_pygame_board(board)

                    turn += 1
                    turn = turn % 2

        if game_over:
            pygame.time.wait(5000)
            pygame.quit()
            quit()
